## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 9
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_9


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 20.5 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Buffalo Bills,8,2022,27.5,54.6,7.9,292.1,97.6,12.5,14.8,...,10.1,104.9,19.625,11.50,12.125,0.096154,2.375,0.625,1.6,2.375
1,Kansas City Chiefs,8,2022,30.4,71.4,7.8,311.8,102.7,11.9,23.6,...,10.7,102.0,21.125,12.25,12.625,0.061350,2.625,0.625,3.4,3.500


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[8, 9]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_9,0.781639,0.300446,0.174067,0.311100,0.395365,0.392523,0.443649
1,Kansas City Chiefs,2022,week_9,0.304284,0.295492,0.184644,0.309466,0.392861,0.297350,0.381789
2,Philadelphia Eagles,2022,week_9,0.436585,0.330753,0.173806,0.337381,0.370765,0.329858,0.378993


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_9,0.781639,0.300446,0.174067,0.311100,0.395365,0.392523,0.443649
1,Kansas City Chiefs,2022,week_9,0.304284,0.295492,0.184644,0.309466,0.392861,0.297350,0.381789
2,Philadelphia Eagles,2022,week_9,0.436585,0.330753,0.173806,0.337381,0.370765,0.329858,0.378993
21,Dallas Cowboys,2022,week_9,0.241246,0.153300,0.090514,0.207929,0.355436,0.209685,0.341162
11,San Francisco 49ers,2022,week_9,0.213895,0.084281,0.080577,0.209658,0.350856,0.187854,0.333736
18,Tampa Bay Buccaneers,2022,week_9,0.172518,0.115002,0.072240,0.191507,0.349401,0.180133,0.327291
4,Miami Dolphins,2022,week_9,0.044133,0.270114,0.111554,0.261138,0.364924,0.210373,0.324825
19,Denver Broncos,2022,week_9,0.083660,0.153300,0.097488,0.199308,0.346480,0.176047,0.313627
9,Cincinnati Bengals,2022,week_9,0.020421,0.129897,0.051104,0.200572,0.346860,0.149771,0.306055
12,Baltimore Ravens,2022,week_9,0.017096,0.032090,0.044273,0.180358,0.343640,0.123491,0.302822
